<a href="https://colab.research.google.com/github/farshidbalan/FluentPython/blob/master/Chapter10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Vector Take #1: Vector2d Compatible

In [0]:
from array import array
import reprlib
import math

In [0]:
class Vector:
  typecode = 'd'
  
  def __init__(self, components):
    # The self._components instance “protected” attribute will hold an array 
    # with the Vector components.
    self._components = array(self.typecode, components)
    
  def __iter__(self):
    # To allow iteration, we return an iterator over self._components
    return iter(self._components)

  def __repr__(self):
    # Use reprlib.repr() to get a limited-length representation of 
    # self._components (e.g., array('d', [0.0, 1.0, 2.0, 3.0, 4.0, ...])).
    components = reprlib.repr(self._components)
    
    # Remove the array('d', prefix and the trailing ) before plugging the string 
    # into a Vector constructor call.
    components = components[components.find('['): -1]
    return f'Vector({components})'
  
  def __bytes__(self):
    # Build a bytes object directly from self._components
    return bytes([ord(self.typecode)]) + bytes(self._components)
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.sqrt(sum(x*x for x in self))
  
  def __boll__(self):
    return bool(abs(self))
  
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    #  we pass the memoryview directly to the constructor
    return cls(memv)

### Remark

The way I used reprlib.repr deserves some elaboration. That function produces safe
representations of large or recursive structures by limiting the length of the output string
and marking the cut with '...'. I wanted the repr of a Vector to look like Vector([3.0,
4.0, 5.0]) and not Vector(array('d', [3.0, 4.0, 5.0])), because the fact that
there is an array inside a Vector is an implementation detail. Because these constructor
calls build identical Vector objects, I prefer the simpler syntax using a list argument.


### Protocol
In the context of object-oriented programming, __a protocol is an informal interface,
defined only in documentation and not in code__. For example, the sequence protocol in
Python entails just the  --len-- and  --getitem-- methods. Any class Spam that imple‐
ments those methods with the standard signature and semantics can be used anywhere
a sequence is expected. Whether Spam is a subclass of this or that is irrelevant; all that

## Vector Take #2: A Sliceable Sequence

In [0]:
class Vector:
  typecode = 'd'
  
  def __init__(self, components):
    # The self._components instance “protected” attribute will hold an array 
    # with the Vector components.
    self._components = array(self.typecode, components)
    
  def __iter__(self):
    # To allow iteration, we return an iterator over self._components
    return iter(self._components)

  def __repr__(self):
    # Use reprlib.repr() to get a limited-length representation of 
    # self._components (e.g., array('d', [0.0, 1.0, 2.0, 3.0, 4.0, ...])).
    components = reprlib.repr(self._components)
    
    # Remove the array('d', prefix and the trailing ) before plugging the string 
    # into a Vector constructor call.
    components = components[components.find('['): -1]
    return f'Vector({components})'
  
  def __bytes__(self):
    # Build a bytes object directly from self._components
    return bytes([ord(self.typecode)]) + bytes(self._components)
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.sqrt(sum(x*x for x in self))
  
  def __boll__(self):
    return bool(abs(self))
  
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    #  we pass the memoryview directly to the constructor
    return cls(memv)
  
  def __len__(self):
    return len(self._components)
  
  
  def __getitem__(self, index):
    return self._components[index]

In [0]:
v1 = Vector([3, 4, 5])
len(v1)

3

In [0]:
v2 = Vector(range(10))

v2[1:4]

array('d', [1.0, 2.0, 3.0])

### Remark

As you can see, even slicing is supported—but not very well. It would be better if a slice
of a Vector was also a Vector instance and not a array. The old FrenchDeck class has
a similar problem: when you slice it, you get a list. In the case of Vector, a lot of
functionality is lost when slicing produces plain arrays.
Consider the built-in sequence types: every one of them, when sliced, produces a new
instance of its own type, and not of some other type.
To make Vector produce slices as Vector instances, we can’t just delegate the slicing to
array. We need to analyze the arguments we get in --getitem-- and do the right thing

In [0]:
import numbers
class Vector:
  typecode = 'd'
  
  def __init__(self, components):
    # The self._components instance “protected” attribute will hold an array 
    # with the Vector components.
    self._components = array(self.typecode, components)
    
  def __iter__(self):
    # To allow iteration, we return an iterator over self._components
    return iter(self._components)

  def __repr__(self):
    # Use reprlib.repr() to get a limited-length representation of 
    # self._components (e.g., array('d', [0.0, 1.0, 2.0, 3.0, 4.0, ...])).
    components = reprlib.repr(self._components)
    
    # Remove the array('d', prefix and the trailing ) before plugging the string 
    # into a Vector constructor call.
    components = components[components.find('['): -1]
    return f'Vector({components})'
  
  def __bytes__(self):
    # Build a bytes object directly from self._components
    return bytes([ord(self.typecode)]) + bytes(self._components)
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.sqrt(sum(x*x for x in self))
  
  def __boll__(self):
    return bool(abs(self))
  
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    
    #  we pass the memoryview directly to the constructor
    return cls(memv)
  
  def __len__(self):
    return len(self._components)
  
  def __getitem__(self, index):
    # Get the class of the instance (i.e., Vector) for later use.
    cls = type(self)
    
    # If the index argument is a slice…
    if isinstance(index, slice):
      
      # …invoke the class to build another Vector instance from a slice of the
      # _components array.
      return cls(self._components[index])
    
    # If the index is an int or some other kind of integer…
    elif isinstance(index, numbers.Integral):
      
      # …just return the specific item from _components. 
      return self._components[index]
    
    else:
      # Otherwise, raise an exception.
      msg = '{cls.__name__} indices must be integers'
      raise TypeError(msg.format(cls=cls))

In [0]:
v3 = Vector(range(7))
v3[-1]

6.0

In [0]:
v3[1:3]

Vector([1.0, 2.0])

In [0]:
v3[1, 4]

TypeError: ignored

## Vector Take #3: Dynamic Attribute Access

### \_\_getattr\_\_ 

“The \_\_getattr\_\_  method is invoked by the interpreter when attribute lookup fails. In
simple terms, given the expression my_obj.x, Python checks if the my_obj instance has
an attribute named x; if not, the search goes to the class (my_obj.\_\_class\_\_ ), and then
up the inheritance graph.2 If the x attribute is not found, then the \_\_getattr\_\_  method
defined in the class of my_obj is called with self and the name of the attribute as a string
(e.g., 'x').

In [0]:
import numbers
class Vector:
  
  shortcut_names = 'xyzt'
  typecode = 'd'
  
  def __init__(self, components):
    # The self._components instance “protected” attribute will hold an array 
    # with the Vector components.
    self._components = array(self.typecode, components)
    
  def __iter__(self):
    # To allow iteration, we return an iterator over self._components
    return iter(self._components)

  def __repr__(self):
    # Use reprlib.repr() to get a limited-length representation of 
    # self._components (e.g., array('d', [0.0, 1.0, 2.0, 3.0, 4.0, ...])).
    components = reprlib.repr(self._components)
    
    # Remove the array('d', prefix and the trailing ) before plugging the string 
    # into a Vector constructor call.
    components = components[components.find('['): -1]
    return f'Vector({components})'
  
  def __bytes__(self):
    # Build a bytes object directly from self._components
    return bytes([ord(self.typecode)]) + bytes(self._components)
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.sqrt(sum(x*x for x in self))
  
  def __boll__(self):
    return bool(abs(self))
  
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    
    #  we pass the memoryview directly to the constructor
    return cls(memv)
  
  def __len__(self):
    return len(self._components)
  
  def __getitem__(self, index):
    # Get the class of the instance (i.e., Vector) for later use.
    cls = type(self)
    
    # If the index argument is a slice…
    if isinstance(index, slice):
      
      # …invoke the class to build another Vector instance from a slice of the
      # _components array.
      return cls(self._components[index])
    
    # If the index is an int or some other kind of integer…
    elif isinstance(index, numbers.Integral):
      
      # …just return the specific item from _components. 
      return self._components[index]
    
    else:
      # Otherwise, raise an exception.
      msg = '{cls.__name__} indices must be integers'
      raise TypeError(msg.format(cls=cls))
      
      
  def __getattr__(self, name):
    cls = type(self)
    if len(name) == 1:
      # Find position of 1-letter name; str.find would also locate 'yz' and we
      # don’t want that, this is the reason for the test above
       pos = cls.shortcut_names.find(name)
       # If the position is within range, return the array element.
       if 0 <= pos < len(self._components):
          return self._components[pos]
        
    # If either test failed, raise AttributeError with a standard message text.
    msg = '{.__name__!r} object has no attribute {!r}'
    raise AttributeError(msg.format(cls, name))

In [0]:
v = Vector(range(7))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [0]:
v.z

2.0

In [0]:
v.k

AttributeError: ignored

In [0]:
v.z = 6

v.f = 9

In [0]:
v, v.f

(Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...]), 9)

### Remark

#### inconsistancy in \_\_getattr\_\_
The inconsistency in Example 10-9 was introduced because of the way \_\_getattr\_\_
works: Python only calls that method as a fall back, when the object does not have the
named attribute. However, after we assign v.x = 10, the v object now has an x attribute,
so \_\_getattr\_\_ will no longer be called to retrieve v.x: the interpreter will just return
the value 10 that is bound to v.x. On the other hand, our implementation of \_\_get
attr\_\_ pays no attention to instance attributes other than self._components, from
where it retrieves the values of the “virtual attributes” listed in shortcut_names.

__We need to customize the logic for setting attributes in our Vector class in order to
avoid this inconsistency.__

In [0]:
import numbers
class Vector:
  
  shortcut_names = 'xyzt'
  typecode = 'd'
  
  def __init__(self, components):
    # The self._components instance “protected” attribute will hold an array 
    # with the Vector components.
    self._components = array(self.typecode, components)
    
  def __iter__(self):
    # To allow iteration, we return an iterator over self._components
    return iter(self._components)

  def __repr__(self):
    # Use reprlib.repr() to get a limited-length representation of 
    # self._components (e.g., array('d', [0.0, 1.0, 2.0, 3.0, 4.0, ...])).
    components = reprlib.repr(self._components)
    
    # Remove the array('d', prefix and the trailing ) before plugging the string 
    # into a Vector constructor call.
    components = components[components.find('['): -1]
    return f'Vector({components})'
  
  def __bytes__(self):
    # Build a bytes object directly from self._components
    return bytes([ord(self.typecode)]) + bytes(self._components)
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  def __abs__(self):
    return math.sqrt(sum(x*x for x in self))
  
  def __boll__(self):
    return bool(abs(self))
  
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    
    #  we pass the memoryview directly to the constructor
    return cls(memv)
  
  def __len__(self):
    return len(self._components)
  
  def __getitem__(self, index):
    # Get the class of the instance (i.e., Vector) for later use.
    cls = type(self)
    
    # If the index argument is a slice…
    if isinstance(index, slice):
      
      # …invoke the class to build another Vector instance from a slice of the
      # _components array.
      return cls(self._components[index])
    
    # If the index is an int or some other kind of integer…
    elif isinstance(index, numbers.Integral):
      
      # …just return the specific item from _components. 
      return self._components[index]
    
    else:
      # Otherwise, raise an exception.
      msg = '{cls.__name__} indices must be integers'
      raise TypeError(msg.format(cls=cls))
      
      
  def __getattr__(self, name):
    cls = type(self)
    if len(name) == 1:
      # Find position of 1-letter name; str.find would also locate 'yz' and we
      # don’t want that, this is the reason for the test above
       pos = cls.shortcut_names.find(name)
       # If the position is within range, return the array element.
       if 0 <= pos < len(self._components):
          return self._components[pos]
        
    # If either test failed, raise AttributeError with a standard message text.
    msg = '{.__name__!r} object has no attribute {!r}'
    raise AttributeError(msg.format(cls, name))
    
    
  def __setattr__(self, name, value):
    cls = type(self)
    
    if len(name) == 1:
      if name in cls.shortcut_names:
        # If name is one of xyzt, set specific error message
        error = 'readonly attribute {attr_name!r}'
      elif name.islower():
        # If name is lowercase, set error message about all single-letter names.
        error = "can't set attributes 'a' to 'z' in {cls_name!r}"
      else:
        error = ''
        
      if error:
        msg = error.format(cls_name=cls.__name__, attr_name=name)
        raise AttributeError(msg)
    # Default case: call __setattr__ on superclass for standard behavior  
    super().__setattr__(name, value)  # remark

### Remark

The super() function provides a way to access methods of super‐
classes dynamically, a necessity in a dynamic language support‐
ing multiple inheritance like Python. It’s used to delegate some task
from a method in a subclass to a suitable method in a superclass,

### Takeaway

Even without supporting writing to the Vector components, here is an important take‐away from 
this example: very often when you implement \_\_getattr\_\_ you need to code
\_\_setattr\_\_ as well, to avoid inconsistent behavior in your objects.
If we wanted to allow changing components, we could implement \_\_setitem\_\_ to en‐
able v[0] = 1.1 and/or \_\_setattr\_\_ to make v.x = 1.1 work. But Vector will remain
immutable because we want to make it hashable in the coming section.

1
3
0
4
1


## Vector Take #4: Hashing and a Faster ==

In [0]:
from array import array
import reprlib
import math
import functools 
import operator
import numbers

class Vector:
  
  shortcut_names = 'xyzt'
  typecode = 'd'
  
  def __init__(self, components):
    # The self._components instance “protected” attribute will hold an array 
    # with the Vector components.
    self._components = array(self.typecode, components)
    
  def __iter__(self):
    # To allow iteration, we return an iterator over self._components
    return iter(self._components)

  def __repr__(self):
    # Use reprlib.repr() to get a limited-length representation of 
    # self._components (e.g., array('d', [0.0, 1.0, 2.0, 3.0, 4.0, ...])).
    components = reprlib.repr(self._components)
    
    # Remove the array('d', prefix and the trailing ) before plugging the string 
    # into a Vector constructor call.
    components = components[components.find('['): -1]
    return f'Vector({components})'
  
  def __bytes__(self):
    # Build a bytes object directly from self._components
    return bytes([ord(self.typecode)]) + bytes(self._components)
  
  def __eq__(self, other):
    return tuple(self) == tuple(other)
  
  
  def __hash__(self):
    
    # Create a generator expression to lazily compute the hash of each component
    hashes = (hash(x) for x in self._components)
    # better to write: map(hash, self._components) 
    
    
    # Feed hashes to reduce with the xor function to compute the aggregate hash
    # value; the third argument, 0, is the initializer
    return functools.reduce(operator.xor, hashes, 0)
  
  def __abs__(self):
    return math.sqrt(sum(x*x for x in self))
  
  def __boll__(self):
    return bool(abs(self))
  
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    
    #  we pass the memoryview directly to the constructor
    return cls(memv)
  
  def __len__(self):
    return len(self._components)
  
  def __getitem__(self, index):
    # Get the class of the instance (i.e., Vector) for later use.
    cls = type(self)
    
    # If the index argument is a slice…
    if isinstance(index, slice):
      
      # …invoke the class to build another Vector instance from a slice of the
      # _components array.
      return cls(self._components[index])
    
    # If the index is an int or some other kind of integer…
    elif isinstance(index, numbers.Integral):
      
      # …just return the specific item from _components. 
      return self._components[index]
    
    else:
      # Otherwise, raise an exception.
      msg = '{cls.__name__} indices must be integers'
      raise TypeError(msg.format(cls=cls))
      
      
  def __getattr__(self, name):
    cls = type(self)
    if len(name) == 1:
      # Find position of 1-letter name; str.find would also locate 'yz' and we
      # don’t want that, this is the reason for the test above
       pos = cls.shortcut_names.find(name)
       # If the position is within range, return the array element.
       if 0 <= pos < len(self._components):
          return self._components[pos]
        
    # If either test failed, raise AttributeError with a standard message text.
    msg = '{.__name__!r} object has no attribute {!r}'
    raise AttributeError(msg.format(cls, name))
    
    
  def __setattr__(self, name, value):
    cls = type(self)
    
    if len(name) == 1:
      if name in cls.shortcut_names:
        # If name is one of xyzt, set specific error message
        error = 'readonly attribute {attr_name!r}'
      elif name.islower():
        # If name is lowercase, set error message about all single-letter names.
        error = "can't set attributes 'a' to 'z' in {cls_name!r}"
      else:
        error = ''
        
      if error:
        msg = error.format(cls_name=cls.__name__, attr_name=name)
        raise AttributeError(msg)
    # Default case: call __setattr__ on superclass for standard behavior  
    super().__setattr__(name, value)  # remark

### Remark

When using reduce, it’s good practice to provide the third argu‐
ment, reduce(function, iterable, initializer), to prevent
this exception: TypeError: reduce() of empty sequence with
no initial value (excellent message: explains the problem and
how to fix it). The initializer is the value returned if the se quence is empty and is used as the first argument in the reducing
loop, so it should be the identity value of the operation. 

### Remark

 in Python 3, map is lazy: it creates a generator that yields the results on demand, thus saving memory—just like the generator expression.